In [14]:
import tensorflow as tf
import pandas as pd
import requests
import json
import base64

In [15]:
data = pd.read_csv('data\Customer-Churn-Records.csv')
inputs = data.iloc[76, :15].to_dict()
inputs

{'RowNumber': 77,
 'CustomerId': 15614049,
 'Surname': 'Hu',
 'CreditScore': 664,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 55,
 'Tenure': 8,
 'Balance': 0.0,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 139161.64,
 'Exited': 0,
 'Complain': 0}

In [16]:
data.iloc[76, :15]

RowNumber                 77
CustomerId          15614049
Surname                   Hu
CreditScore              664
Geography             France
Gender                  Male
Age                       55
Tenure                     8
Balance                  0.0
NumOfProducts              2
HasCrCard                  1
IsActiveMember             1
EstimatedSalary    139161.64
Exited                     0
Complain                   0
Name: 76, dtype: object

In [18]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://bcc-prediction-production.up.railway.app/v1/models/bcc-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "Non Active" if prediction_value < 0.5 else "Active"
else:
    result = "Error: No predictions found in response."

print(result)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)